# Inicialización




In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from google.colab import drive

import tensorflow as tf
from tensorflow import keras
from keras import layers

from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import SGD

from sklearn.preprocessing import MinMaxScaler

random_state = 33
np.random.seed(random_state)        # Semilla para NumPy
tf.random.set_seed(random_state)    # Semilla para TensorFlow

In [ ]:
#Para acceder a los ficheros de Google Drive
drive.mount('/content/drive')
# La carpeta datos (que debe contiene los archivos de datos) debe estar en vuestro Drive, dentro de la carpeta 'Colab Notebooks'

#####métricas con el valor fitness

In [ ]:
def valor_fitness(instancia, caso):
  instancia_local = instancia.copy()
  valores = caso[2::2]
  pesos = caso[1::2]
  pesoAct = np.sum(pesos*instancia_local)
  i=0
  while(pesoAct > caso[0]): # mientras el peso sea mayor
    if(instancia_local[i]):
      pesoAct -= pesos[i]
      instancia_local[i] = 0
    i += 1
  return np.sum(valores*instancia_local)


In [ ]:
def metrica_valor_fitness(y_pred, x):
  sum_valores = 0
  num_tot = len(x)
  num_items = int(len(x[0])/2+1)
  print(num_items)
  umbral = 0.5
  predictions_binary = (y_pred >= umbral).astype(int)
  for i in range(num_tot):
    sum_valores += valor_fitness(predictions_binary[i][ num_items:], x[i])
  return sum_valores/num_tot

In [ ]:
def metrica_valor_fitness(predictions_binary, x):
  sum_valores = 0
  num_tot = len(x)
  num_items = int(len(x[0])/2+1)
  for i in range(num_tot):
    sum_valores += valor_fitness(predictions_binary[i][ num_items:], x[i])
  return sum_valores/num_tot

#####Métrica fitness comparando con la solucion optima

In [ ]:
def metrica_fitness_mejor_solucion(predictions, input, exact_sols):
  sum_pred = 0
  sum_exact = 0
  preds = predictions[:,int(predictions.shape[1] / 2)+1:].copy()
  for caso in range(len(input)):
    valores = input[caso][2::2]
    pesos = input[caso][1::2]
    pesoAct = np.sum(pesos*preds[caso])
    i=0
    while(pesoAct > input[caso][0]): # mientras el peso sea mayor
      if(preds[caso][i]):
        pesoAct -= pesos[i]
        preds[caso][i] = 0
      i += 1
    sum_pred += np.sum(valores*preds[caso])
    sum_exact += exact_sols[caso]
    # if caso %1000 == 0: print(caso)
  return sum_pred/sum_exact

#Experimento Mochila $N=10$ y $10^5$ casos, con solucion optima

Debido al alto coste computacional de entrenar las redes neuronales, ejecutar etse experimento lleva una cantidad considerable de horas.

El problema utiliza $N$ objetos exactamente y los datos se cargan desde un fichero .csv como un vector de X casos y cada posición es un vector de $4N+2$ variables que representan:   


0   Número de Objetos.  
1   Capacidad de la mochila.   
2 - 2N+1 Peso y Valor de cada objeto en ese orden.  

Voy a utilizar el propio caso como etiqueta, aunque no utilice la etiqueta para nada más que evaluar la solución generada.




He generado un conjunto de posibles casos para el problema y con sus soluciones exactas calculadas para cada caso.

###Cargar y separar datos

In [ ]:
file_path = 'drive/MyDrive/Colab Notebooks/datos/TFG/data1_10_10_5Comas.csv'

datos = np.genfromtxt(file_path, delimiter=",") #Funcion cargar datos desde txt, y en este caso desde csv

numItems = datos[0][0]
x = datos[:, 1:int(numItems*2+2)]  # datos de entrada
y = datos[:, 1:int(numItems*2+2)]   # datos de etiqueta
x = np.array(x, np.float64)
y = np.array(y, np.float64)
exact_sols = datos[:,-1]
print("Número de objetos (N) =", numItems)
# Mostrar dimension del conjunto de muestras total
print("Forma de vector X de muestras:", x.shape)
print("Forma de vector Y de etiquetas:", y.shape)
print("Forma de vector exact_sols de etiquetas:", exact_sols.shape)

print("Ejemplos:")
print("Datos = ",datos[0])
print("exact_sols = ",exact_sols[0])
print("X = ",x[0])
print("Y = ",y[0])

Separo el conjunto de casos en test y train para evaluar los modelos utilizados

In [ ]:
#Separate data in test and train

np.random.seed(0)
trainPortion = 0.8 #porcentaje de train, el porcentaje de test será la resta de 1 menos el porcentaje de train

#-------------Obtener índices

indexesData = np.arange(len(y)) #Indices del conjunto de muestras
#-------------Desordenar indices y separar en rain y test

np.random.shuffle(indexesData) #Desordenar indices de las muestras
numberTrain = round(len(indexesData)*trainPortion) #numero de muestras para train
trainIndexes = indexesData[:numberTrain]
testIndexes = indexesData[numberTrain:]

#-------------Datos desorden:ados para train y test

trainX = x[trainIndexes]
testX = x[testIndexes]
trainY = y[trainIndexes]
testY = y[testIndexes]

#-------------Copia de los datos para mantener datos originales después del preprocesado

original_trainX = trainX.copy()
original_testX = testX.copy()
original_trainY = trainY.copy()
original_testY = testY.copy()

#-------------Mostrar resultados

print('Muestras totales:  {}'.format(len(trainY)+len(testX)))
print('Muestras train:  {}'.format(len(trainY)))
print('Muestras test:  {}'.format(len(testX)))


## Preprocesamiento de los datos

Como los datos han sido generados artificialmente y no se han obtenido de fuentes externas, no es necesario realizar la búsqueda de datos faltantes o erroneos ni desbalanceo en las proporciones ya que los datos han sido generados teniendo en cuenta estos factores. La codificación de los datos también se ha tenido en cuenta en al generación de los datos.

### Comprobación de Duplicados en los datos

In [ ]:
# Comprobando duplicados en datos
unique_rows= np.unique(datos, axis=0)
num_duplicates = datos.shape[0] - unique_rows.shape[0]

print("Número de filas duplicadas en los datos:", num_duplicates)

### Escalar Datos

In [ ]:
# Crear el objeto MinMaxScaler
scaler = MinMaxScaler()

# Ajustar el escalador con los datos de entrenamiento y transformar tanto train como test
trainX_scaled = scaler.fit_transform(trainX)
testX_scaled = scaler.transform(testX)
trainX_scaled = np.array(trainX_scaled)
testX_scaled = np.array(testX_scaled)

print("train X original = ",trainX[0])
print("train X scaled = ",trainX_scaled[0])
print("test X = ",testX_scaled[0])

## Funciones de Pérdida Personalizada

Versiones iniciales de la función de pérdida

In [ ]:
def fitness_loss_custom(target, output):
    target_0 = tf.gather(target, indices=0, axis=1)
    valores = target[:, 2::2]
    pesos = target[:, 1::2]
    pred = output[:, int(output.shape[1] / 2)+1:]

    # Inicializar el peso acumulado y la máscara
    peso_act = tf.reduce_sum(pesos * pred, axis=1)
    mask = tf.ones_like(valores)

    # Condición para el bucle while: peso_act > target_0
    def cond(i, peso_act, mask):
        return tf.reduce_any(peso_act > target_0)

    # Cuerpo del bucle while
    def body(i, peso_act, mask):
        # Actualizar peso_act y mask
        update_mask = tf.where(pred[:, i] > 0, tf.zeros_like(mask[:, i]), mask[:, i])
        update_peso_act = tf.where(pred[:, i] > 0, peso_act - pesos[:, i], peso_act)

        return [i + 1, update_peso_act, tf.concat([mask[:, :i], tf.expand_dims(update_mask, -1), mask[:, i+1:]], axis=1)]

    # Aplicar el bucle while
    i = tf.constant(0)
    [_, peso_act, mask] = tf.while_loop(cond, body, loop_vars=[i, peso_act, mask], shape_invariants=[i.get_shape(), peso_act.get_shape(), tf.TensorShape([None, None])])

    # Calcular la pérdida final
    final_loss = 1.0 / tf.reduce_sum(valores * mask * pred, axis=1)

    return final_loss


In [ ]:
import tensorflow as tf

def fitness_loss_custom2(target, output):
    # Función que se aplicará a cada par (target_i, output_i)
    def process_instance(args):
        #print(args)
        target_i, output_i = args
        target_0_i = target_i[0]
        valores_i = target_i[2::2]
        pesos_i = target_i[1::2]
        pred_i = output_i[int(output_i.shape[0] / 2)+1:]

        mask_i = tf.ones_like(valores_i)
        # Funciones para condiciones y cuerpo del bucle while
        def cond(peso_act, i):
            return tf.reduce_any(peso_act > target_0_i)

        def body(peso_act, i):
            update_mask = tf.where(pred_i[i] > 0, 0., mask_i[i])
            update_peso_act = tf.where(pred_i[i] > 0, peso_act - pesos_i[i], peso_act)
            #print(i)
            return [update_peso_act, i + 1]

        peso_act = tf.reduce_sum(pesos_i * pred_i)
        #print(peso_act)
        #print(target_0_i)
        i = tf.constant(0)
        # Bucle while en TensorFlow
        peso_act, _ = tf.while_loop(cond, body, loop_vars=[peso_act, i])

        final_loss_i = 1.0 / tf.reduce_sum(valores_i * mask_i * pred_i)
        return final_loss_i

    # Aplicar 'process_instance' a cada par (target_i, output_i) usando map_fn
    final_losses = tf.map_fn(process_instance, (target, output), dtype=tf.float32)
    return final_losses

# Dummy data para ejemplificar la compilación
num_items = 10  # Supongamos que cada instancia tiene 10 items
batch_size = 32
dummy_target = tf.convert_to_tensor(np.array(trainY[:batch_size], dtype=np.float64),dtype=tf.float32)
dummy_output = tf.random.uniform([batch_size, 2 * num_items + 1],minval=0.99, maxval=1.0)
#print(dummy_target[0], dummy_output[0])
# Ejemplo de llamada a la función
loss_values = fitness_loss_custom2(dummy_target, dummy_output)
#print(loss_values)


Esta función de pérdida `fitness_loss_custom3` es la función de pérdida finalmente usada comprobando que funcione como se espera.

In [ ]:
import tensorflow as tf
import keras
def fitness_loss_custom3(target, output):
    # Función que se aplicará a cada par (target_i, output_i)
    def process_instance(args):
        #print(args)
        target_i, output_i = args
        target_0_i = target_i[0]
        valores_i = target_i[2::2]
        pesos_i = target_i[1::2]
        pred_i = output_i[int(output_i.shape[0] / 2)+1:]

        mask_i = tf.ones_like(valores_i)
        # Funciones para condiciones y cuerpo del bucle while
        def cond(peso_act, i, mask_i):
            return tf.reduce_any(peso_act > target_0_i)

        def body(peso_act, i, mask_i):
            update_mask = tf.where(pred_i[i] > 0, 0., mask_i[i])
            update_peso_act = tf.where(pred_i[i] > 0, peso_act - pesos_i[i], peso_act)
            #print(i)
            return [update_peso_act, i + 1, tf.tensor_scatter_nd_update(mask_i, [[i]], [update_mask])]

        peso_act = tf.reduce_sum(pesos_i * pred_i)
        #print(peso_act)
        #print(target_0_i)
        # print(peso_act,valores_i,pred_i,mask_i)
        i = tf.constant(0)
        # Bucle while en TensorFlow
        [peso_act, i ,  mask_i] = tf.while_loop(cond, body, loop_vars=[peso_act, i,mask_i])
        # print(peso_act,mask_i)

        final_loss_i = tf.reduce_sum(valores_i * mask_i * pred_i)
        # print(final_loss_i)
        return final_loss_i

    # Aplicar 'process_instance' a cada par (target_i, output_i) usando map_fn
    final_losses = tf.map_fn(process_instance, (target, output), fn_output_signature=tf.float32)

    return 100.0 / tf.reduce_sum(final_losses)

# Dummy data para ejemplificar la compilación
num_items = 10  # Supongamos que cada instancia tiene 10 items
batch_size = 32
dummy_target = tf.convert_to_tensor(np.array(trainY[:batch_size], dtype=np.float64),dtype=tf.float32)
dummy_output = tf.random.uniform([batch_size, 2 * num_items + 1],minval=0.99, maxval=1.0)
#print(dummy_target[0], dummy_output[0])
# Ejemplo de llamada a la función
loss_values = fitness_loss_custom3(dummy_target, dummy_output)
print(loss_values)
bce = keras.losses.BinaryCrossentropy(from_logits=True)
bce(dummy_target, dummy_output)

Esta funcion calcula exactamente el valor fitness de cada muestra por separado redondeando las probabilidades de la red neuronal para obtener la solucion que ofrece. Se ha observado que no funciona correctamente con la red neuronal debido al caracter probabilistico  y no entero de las redes neuronales para clasificación.

In [ ]:
import tensorflow as tf

def fitness_loss_custom_entera(target, output):
    # Función que se aplicará a cada par (target_i, output_i)
    def process_instance(args):
        #print(args)
        target_i, output_i = args
        target_0_i = target_i[0]
        valores_i = target_i[2::2]
        pesos_i = target_i[1::2]
        pred_i = output_i[int(output_i.shape[0] / 2)+1:]

        #mask_i = tf.ones_like(valores_i)
        mask_i = tf.where(pred_i < 0.5, 0., 1.)
        # Funciones para condiciones y cuerpo del bucle while
        def cond(peso_act, i, mask_i):
            return tf.reduce_any(peso_act > target_0_i)

        def body(peso_act, i, mask_i):
            update_mask = tf.where(pred_i[i] > 0, 0., mask_i[i])
            update_peso_act = tf.where(pred_i[i] > 0, peso_act - pesos_i[i], peso_act)
            #print(i)
            return [update_peso_act, i + 1, tf.tensor_scatter_nd_update(mask_i, [[i]], [update_mask])]

        peso_act = tf.reduce_sum(pesos_i * mask_i)
        #print(peso_act)
        #print(target_0_i)
        #print(peso_act,valores_i,pesos_i,pred_i,mask_i)
        i = tf.constant(0)
        # Bucle while en TensorFlow
        [peso_act, i ,  mask_i] = tf.while_loop(cond, body, loop_vars=[peso_act, i,mask_i])
        #print(peso_act,mask_i)

        final_loss_i = tf.reduce_sum(valores_i * mask_i)
        #print(final_loss_i)
        return final_loss_i

    # Aplicar 'process_instance' a cada par (target_i, output_i) usando map_fn
    final_losses = tf.map_fn(process_instance, (target, output), fn_output_signature=tf.float32)

    return 100.0 / tf.reduce_sum(final_losses)

# Dummy data para ejemplificar la compilación
num_items = 10  # Supongamos que cada instancia tiene 10 items
batch_size = 32
dummy_target = tf.convert_to_tensor(np.array(trainY[:batch_size], dtype=np.float64),dtype=tf.float32)
dummy_output = tf.random.uniform([batch_size, 2 * num_items + 1],minval=0.0, maxval=1.0)
#print(dummy_target[0], dummy_output[0])
# Ejemplo de llamada a la función
loss_values = fitness_loss_custom_entera(dummy_target, dummy_output)
print(loss_values)
bce = keras.losses.BinaryCrossentropy(from_logits=True)
bce(dummy_target, dummy_output)

## Creación del Modelo

Valores de entrada y salida de los modelos:

In [ ]:
input_shape  = 2*numItems +1
output_shape = input_shape  # Las etiquetas incluyen la instancia del problema
dense_neurons  = (numItems*numItems*numItems,numItems*numItems*numItems,numItems*numItems*5,numItems*numItems*2)

Vamos a implementar una función que nos cree el modelo que vamos a entrenar y evaluar.

In [ ]:
def create_model(dense_neurons, input_shape, output_shape):
    local_model = keras.Sequential()

    # Capa de entrada
    local_model.add(layers.Input(shape=(int(input_shape),)))

    # capas ocultas con activación Relu
    for neurons in dense_neurons:
      local_model.add(layers.Dense(neurons, activation='relu'))

    # Capa de salida para clasificación con tantas neuronas como salidas esperadas
    local_model.add(layers.Dense(output_shape, activation='sigmoid'))
    return local_model

Función para crear un modelo con dropout

In [ ]:
def create_model_dropout(dense_neurons, input_shape, output_shape):
    local_model = keras.Sequential()

    # Capa de entrada
    local_model.add(layers.Input(shape=(int(input_shape),)))

    # capas ocultas con activación Relu
    for neurons in dense_neurons:
      local_model.add(layers.Dense(neurons, activation='relu'))
      local_model.add(layers.Dropout(0.5))

    # Capa de salida para clasificación con tantas neuronas como salidas esperadas
    local_model.add(layers.Dense(output_shape, activation='sigmoid'))
    return local_model

Función para crear un modelo con normalización por lotes

In [ ]:
def create_model_normalization(dense_neurons, input_shape, output_shape):
    local_model = keras.Sequential()

    # Capa de entrada
    local_model.add(layers.Input(shape=(int(input_shape),)))

    # Capas ocultas con activación ReLU
    for neurons in dense_neurons:
        local_model.add(layers.Dense(neurons))
        local_model.add(layers.BatchNormalization())  # Añade Batch Normalization
        local_model.add(layers.Activation('relu'))    # Aplica la activación después de normalizar

    # Capa de salida para clasificación con tantas neuronas como salidas esperadas
    local_model.add(layers.Dense(output_shape, activation='sigmoid'))

    return local_model

## Grid Search

Definimos la función para utilizar grid_search

In [ ]:
def single_grid_search(dense_neurons, input_shape, output_shape, X_scaled,X, Y, exact_sols, epochs, batch_size, learn_rate, _n_splits = 5, _random_state = 33):

    # Inicializar KFold
    kfold = KFold(n_splits=_n_splits, shuffle=True, random_state=_random_state)

    loss_results = []
    mejora_results = []
    fitness_results = []

    # Iterar sobre cada fold (validación cruzada)
    for train_index, test_index in kfold.split(X_scaled):


        # Dividir datos en entrenamiento y prueba
        trainX_scaled, valX_scaled = X_scaled[train_index], X_scaled[test_index]
        trainX, valX = X[train_index], X[test_index]
        trainY, valY = Y[train_index], Y[test_index]

        # Crear modelo
        local_model = create_model(dense_neurons, input_shape, output_shape)

        # Compilar el modelo
        sgd_optimizer = SGD(learning_rate=learn_rate)
        local_model.compile(optimizer=sgd_optimizer,
                            loss = fitness_loss_custom3)

        # Entrenar el modelo
        local_model.fit(
            trainX_scaled, trainY,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
        )

        # Evaluar el modelo
        scores = local_model.evaluate(valX_scaled, valY, verbose=0)
        loss_results.append(scores)


        # Evaluar metricas personalizadas
        y_pred = local_model.predict(valX_scaled,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)

        mejora_results.append(metrica_fitness_mejor_solucion(y_pred_binary, valX, exact_sols[test_index]))
        fitness_results.append(metrica_valor_fitness(y_pred_binary, valX))

    # Devolver el promedio de las evaluaciones
    return np.mean(loss_results), np.std(loss_results),np.mean(mejora_results),np.mean(fitness_results)

Valores de los hiperparámetros a probar:

In [ ]:
# Valores de los hiperparámetros para el modelo base
dense_neurons  = (numItems*numItems*numItems,numItems*numItems*numItems,numItems*numItems*5,numItems*numItems*2)
batch_size = 64
learning_rate = 0.01
epochs = 40
k_folds = 5


# Valores de los  distintos hiperparámetros
learning_rate_search  = [0.1,0.01,0.001,0.0001]
batch_size_search = [16,32,64,128]

> Hiperparámetro de la tasa de aprendizaje (learning rate)

In [ ]:
loss_mean_table = []
loss_std_table = []
mejora_table = []
fitness_table = []

for lr in learning_rate_search:
    # Entrenar y evaluar
    results = single_grid_search(dense_neurons, input_shape, output_shape, trainX_scaled, trainX, trainY, datos[trainIndexes,-1], epochs, batch_size, lr, k_folds, random_state)

    loss_mean_table.append(results[0])
    loss_std_table.append(results[1])
    mejora_table.append(results[2])
    fitness_table.append(results[3])


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Tasa de Aprendizaje': learning_rate_search,'Error de validación medio (función de pérdida)':loss_mean_table,'Desviación del error de validación (función de pérdida)':loss_std_table,'Mejora':mejora_table,'Fitness':fitness_table})
display(tableFrame)

> Hiperparámetro del tamaño de lote (batch size)

In [ ]:
loss_mean_table = []
loss_std_table = []
mejora_table = []
fitness_table = []

for bs in batch_size_search:
    # Entrenar y evaluar
    results = single_grid_search(dense_neurons, input_shape, output_shape, trainX_scaled, trainX, trainY, datos[trainIndexes,-1], epochs, bs, learning_rate, k_folds, random_state)

    loss_mean_table.append(results[0])
    loss_std_table.append(results[1])
    mejora_table.append(results[2])
    fitness_table.append(results[3])


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Tasa de Aprendizaje': batch_size_search,'Error de validación medio (función de pérdida)':loss_mean_table,'Desviación del error de validación (función de pérdida)':loss_std_table,'Mejora':mejora_table,'Fitness':fitness_table})
display(tableFrame)

## Modelo Final

In [ ]:
# Valores de los hiperparámetros para el modelo final
dense_neurons  = (numItems*numItems*numItems,numItems*numItems*numItems,numItems*numItems*5,numItems*numItems*2)
batch_size = 64
epochs = 40
learning_rate = 0.01



final_model = create_model(dense_neurons, input_shape, output_shape)
# Compilar el modelo
sgd_optimizer = SGD(learning_rate=learning_rate)
final_model.compile(optimizer=sgd_optimizer,
                    loss=fitness_loss_custom3)
#print(final_model.summary())

final_model.fit(
    trainX_scaled, trainY,
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

## Resultados

In [ ]:
fitness_table = []
mejora_table = []
loss_mean_table = []

# Evaluar el modelo
scores = final_model.evaluate(testX_scaled, testY, verbose=0)
loss_mean_table.append(scores)

# Evaluar metricas personalizadas
y_pred = final_model.predict(testX_scaled,verbose=0)

umbral = 0.5
y_pred_binary = (y_pred >= umbral).astype(int)
mejora_table.append(metrica_fitness_mejor_solucion(y_pred_binary, testX,  datos[testIndexes,-1]))
fitness_table.append(metrica_valor_fitness(y_pred_binary, testX))



#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Error de validación (función de pérdida)':loss_mean_table,'Mejora':mejora_table,'Fitness':fitness_table})
display(tableFrame)

## Curvas de Aprendizaje Función de Pérdida Probabilistica

In [ ]:
def create_learning_curves_mejora(curves_model, train_X_scaled, train_X, train_Y, val_X_scaled, val_X, val_Y, exact_sols_train, exact_sols_val, batch_size,epochs, iters):
    loss_results = [[],[]]
    mejora_results = [[],[]]
    fitness_results = [[],[]]

    for i in range(iters):
        # Entrenar el modelo
        curves_model.fit(
            train_X_scaled, train_Y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=2,
        )

        e_type = 0 # Entrenameinto
        # Evaluar el modelo
        scores = curves_model.evaluate(train_X_scaled, train_Y, verbose=0)
        loss_results[e_type].append(scores)

        # Evaluar metricas personalizadas
        y_pred = curves_model.predict(train_X_scaled,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)

        mejora_results[e_type].append(metrica_fitness_mejor_solucion(y_pred_binary, train_X, exact_sols_train))
        fitness_results[e_type].append(metrica_valor_fitness(y_pred_binary, train_X))


        e_type = 1 # Validación
        # Evaluar el modelo
        scores = curves_model.evaluate(val_X_scaled, val_Y, verbose=0)
        loss_results[e_type].append(scores)

        # Evaluar metricas personalizadas
        y_pred = curves_model.predict(val_X_scaled,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)





        mejora_results[e_type].append(metrica_fitness_mejor_solucion(y_pred_binary, val_X, exact_sols_val))
        fitness_results[e_type].append(metrica_valor_fitness(y_pred_binary, val_X))

    return loss_results, mejora_results, fitness_results,


dense_neurons  = (numItems*numItems*numItems,numItems*numItems*numItems,numItems*numItems*5,numItems*numItems*2)
batch_size = 64
learning_rate = 0.01
curves_model = create_model(dense_neurons, input_shape, output_shape)

# Compilar el modelo
sgd_optimizer = SGD(learning_rate=learning_rate)
curves_model.compile(optimizer=sgd_optimizer,
                    loss=fitness_loss_custom3)

epochs = 1
iters = 40
curves_results = create_learning_curves_mejora(curves_model, trainX_scaled, trainX, trainY,testX_scaled,testX,testY,datos[trainIndexes,-1], datos[testIndexes,-1], batch_size, epochs,iters)

### Función de Pérdida

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje de la Función de Perdida",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Función de Pérdida",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[0][0], 'b',label=r'$Pérdida_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[0][1], 'orange',label=r'$Pérdida_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

### Valor Fitness

In [ ]:
plt.figure(figsize=(12, 7))
plt.title(r"Curva de Aprendizaje del Valor de $Fitness$",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel(r"Valor $Fitness$",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[2][0], 'b',label=r'$Fitness_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[2][1], 'orange',label=r'$Fitness_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

### Ratio de mejora soluciones exactas

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje de la Mejora respecto de las Soluciones Exactas",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Ratio de Mejora",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[1][0], 'b',label=r'$Mejora_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[1][1], 'orange',label=r'$Mejora_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

## Curvas de Aprendizaje Función de Pérdida Entera

In [ ]:
def create_learning_curves_mejora(curves_model, train_X_scaled, train_X, train_Y, val_X_scaled, val_X, val_Y, exact_sols_train, exact_sols_val, batch_size,epochs, iters):
    loss_results = [[],[]]
    mejora_results = [[],[]]
    fitness_results = [[],[]]

    for i in range(iters):
        # Entrenar el modelo
        curves_model.fit(
            train_X_scaled, train_Y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
        )

        e_type = 0 # Entrenameinto
        # Evaluar el modelo
        scores = curves_model.evaluate(train_X_scaled, train_Y, verbose=0)
        loss_results[e_type].append(scores)

        # Evaluar metricas personalizadas
        y_pred = curves_model.predict(train_X_scaled,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)

        mejora_results[e_type].append(metrica_fitness_mejor_solucion(y_pred_binary, train_X, exact_sols_train))
        fitness_results[e_type].append(metrica_valor_fitness(y_pred_binary, train_X))


        e_type = 1 # Validación
        # Evaluar el modelo
        scores = curves_model.evaluate(val_X_scaled, val_Y, verbose=0)
        loss_results[e_type].append(scores)

        # Evaluar metricas personalizadas
        y_pred = curves_model.predict(val_X_scaled,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)





        mejora_results[e_type].append(metrica_fitness_mejor_solucion(y_pred_binary, val_X, exact_sols_val))
        fitness_results[e_type].append(metrica_valor_fitness(y_pred_binary, val_X))

    return loss_results, mejora_results, fitness_results,


dense_neurons  = (numItems*numItems*numItems,numItems*numItems*numItems,numItems*numItems*5,numItems*numItems*2)
batch_size = 64
learning_rate = 0.01
curves_model_entera = create_model(dense_neurons, input_shape, output_shape)

# Compilar el modelo
sgd_optimizer = SGD(learning_rate=learning_rate)
curves_model_entera.compile(optimizer=sgd_optimizer,
                    loss=fitness_loss_custom_entera)

epochs = 1
iters = 40
curves_results_entera = create_learning_curves_mejora(curves_model_entera, trainX_scaled, trainX, trainY,testX_scaled,testX,testY,datos[trainIndexes,-1], datos[testIndexes,-1], batch_size, epochs,iters)

### Función de Pérdida

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje de la Función de Perdida",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Función de Pérdida",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_entera[0][0], 'b',label=r'$Pérdida_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_entera[0][1], 'orange',label=r'$Pérdida_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

### Valor Fitness

In [ ]:
plt.figure(figsize=(12, 7))
plt.title(r"Curva de Aprendizaje del Valor de $Fitness$",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel(r"Valor $Fitness$",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_entera[2][0], 'b',label=r'$Fitness{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_entera[2][1], 'orange',label=r'$Fitness{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

### Ratio de mejora soluciones exactas

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje de la Mejora respecto de las Soluciones Exactas",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Ratio de Mejora",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_entera[1][0], 'b',label=r'$Mejora_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_entera[1][1], 'orange',label=r'$Mejora_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

## Curvas de Aprendizaje Función de Pérdida Batch Normalization

In [ ]:
def create_learning_curves_mejora(curves_model, train_X_scaled, train_X, train_Y, val_X_scaled, val_X, val_Y, exact_sols_train, exact_sols_val, batch_size,epochs, iters):
    loss_results = [[],[]]
    mejora_results = [[],[]]
    fitness_results = [[],[]]

    for i in range(iters):
        # Entrenar el modelo
        curves_model.fit(
            train_X_scaled, train_Y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
        )

        e_type = 0 # Entrenameinto
        # Evaluar el modelo
        scores = curves_model.evaluate(train_X_scaled, train_Y, verbose=0)
        loss_results[e_type].append(scores)

        # Evaluar metricas personalizadas
        y_pred = curves_model.predict(train_X_scaled,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)

        mejora_results[e_type].append(metrica_fitness_mejor_solucion(y_pred_binary, train_X, exact_sols_train))
        fitness_results[e_type].append(metrica_valor_fitness(y_pred_binary, train_X))


        e_type = 1 # Validación
        # Evaluar el modelo
        scores = curves_model.evaluate(val_X_scaled, val_Y, verbose=0)
        loss_results[e_type].append(scores)

        # Evaluar metricas personalizadas
        y_pred = curves_model.predict(val_X_scaled,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)





        mejora_results[e_type].append(metrica_fitness_mejor_solucion(y_pred_binary, val_X, exact_sols_val))
        fitness_results[e_type].append(metrica_valor_fitness(y_pred_binary, val_X))

    return loss_results, mejora_results, fitness_results,


dense_neurons  = (numItems*numItems*numItems,numItems*numItems*numItems,numItems*numItems*5,numItems*numItems*2)
batch_size = 64
learning_rate = 0.01
curves_model_norm = create_model_normalization(dense_neurons, input_shape, output_shape)


# Compilar el modelo
sgd_optimizer = SGD(learning_rate=learning_rate)
curves_model_norm.compile(optimizer=sgd_optimizer,
                    loss=fitness_loss_custom3)

print(curves_model_norm.summary())

epochs = 1
iters = 40
curves_results_norm = create_learning_curves_mejora(curves_model_norm, trainX_scaled, trainX, trainY,testX_scaled,testX,testY,datos[trainIndexes,-1], datos[testIndexes,-1], batch_size, epochs,iters)

### Función de Pérdida

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje de la Función de Perdida",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Función de Pérdida",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_norm[0][0], 'b',label=r'$Pérdida_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_norm[0][1], 'orange',label=r'$Pérdida_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

### Valor Fitness

In [ ]:
plt.figure(figsize=(12, 7))
plt.title(r"Curva de Aprendizaje del Valor de $Fitness$",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel(r"Valor $Fitness$",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_norm[2][0], 'b',label=r'$Fitness_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_norm[2][1], 'orange',label=r'$Fitness_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

### Ratio de mejora soluciones exactas

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje de la Mejora respecto de las Soluciones Exactas",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Ratio de Mejora",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_norm[1][0], 'b',label=r'$Mejora_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_norm[1][1], 'orange',label=r'$Mejora_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

## Curvas de Aprendizaje Función de Pérdida Dropout

In [ ]:
def create_learning_curves_mejora(curves_model, train_X_scaled, train_X, train_Y, val_X_scaled, val_X, val_Y, exact_sols_train, exact_sols_val, batch_size,epochs, iters):
    loss_results = [[],[]]
    mejora_results = [[],[]]
    fitness_results = [[],[]]

    for i in range(iters):
        # Entrenar el modelo
        curves_model.fit(
            train_X_scaled, train_Y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=2,
        )

        e_type = 0 # Entrenameinto
        # Evaluar el modelo
        scores = curves_model.evaluate(train_X_scaled, train_Y, verbose=0)
        loss_results[e_type].append(scores)

        # Evaluar metricas personalizadas
        y_pred = curves_model.predict(train_X_scaled,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)

        mejora_results[e_type].append(metrica_fitness_mejor_solucion(y_pred_binary, train_X, exact_sols_train))
        fitness_results[e_type].append(metrica_valor_fitness(y_pred_binary, train_X))


        e_type = 1 # Validación
        # Evaluar el modelo
        scores = curves_model.evaluate(val_X_scaled, val_Y, verbose=0)
        loss_results[e_type].append(scores)

        # Evaluar metricas personalizadas
        y_pred = curves_model.predict(val_X_scaled,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)





        mejora_results[e_type].append(metrica_fitness_mejor_solucion(y_pred_binary, val_X, exact_sols_val))
        fitness_results[e_type].append(metrica_valor_fitness(y_pred_binary, val_X))

    return loss_results, mejora_results, fitness_results,


dense_neurons  = (numItems*numItems*numItems,numItems*numItems*numItems,numItems*numItems*5,numItems*numItems*2)
batch_size = 64
learning_rate = 0.01
curves_model_dropout = create_model_dropout(dense_neurons, input_shape, output_shape)


# Compilar el modelo
sgd_optimizer = SGD(learning_rate=learning_rate)
curves_model_dropout.compile(optimizer=sgd_optimizer,
                    loss=fitness_loss_custom3)

print(curves_model_dropout.summary())

epochs = 1
iters = 40
curves_results_dropout = create_learning_curves_mejora(curves_model_dropout, trainX_scaled, trainX, trainY,testX_scaled,testX,testY,datos[trainIndexes,-1], datos[testIndexes,-1], batch_size, epochs,iters)

### Función de Pérdida

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje de la Función de Perdida",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Función de Pérdida",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_dropout[0][0], 'b',label=r'$Pérdida_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_dropout[0][1], 'orange',label=r'$Pérdida_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

### Valor Fitness

In [ ]:
plt.figure(figsize=(12, 7))
plt.title(r"Curva de Aprendizaje del Valor de $Fitness$",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel(r"Valor $Fitness$",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_dropout[2][0], 'b',label=r'$Fitness_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_dropout[2][1], 'orange',label=r'$Fitness_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

### Ratio de mejora soluciones exactas

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje de la Mejora respecto de las Soluciones Exactas",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Ratio de Mejora",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_dropout[1][0], 'b',label=r'$Mejora_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results_dropout[1][1], 'orange',label=r'$Mejora_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()